<a href="https://colab.research.google.com/github/nermienkh/POC-Text-2-Text-Chatbot/blob/main/QAModels_Arabic_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#install needed dependencies

In [1]:
!nvidia-smi

Wed Jul  5 12:15:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install datasets
!pip install transformers==4.3.3
!pip install sentencepiece
#only needed for AraBERTv1 and v2
!pip install farasapy
!pip install pyarabic
!pip install fuzzysearch

#Get QA code from the transformers library `transformers==4.3.3`

In [ ]:
!git clone https://github.com/aub-mind/arabert
!cp arabert/examples/question-answering/utils_qa.py .
!cp arabert/examples/question-answering/trainer_qa.py .
!cp arabert/examples/question-answering/run_qa.py .
!cp arabert/examples/question-answering/squad_preprocessing.py .

Download the TyDiQA data

In [ ]:
!wget https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-train.json
!wget https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-dev.json

--2021-01-25 07:11:15--  https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-train.json
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.121.128, 74.125.126.128, 74.125.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.121.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58004076 (55M) [application/json]
Saving to: ‘tydiqa-goldp-v1.1-train.json.1’

tydiqa-goldp-v1.1-t 100%[===================>]  55.32M   245MB/s    in 0.2s    

2021-01-25 07:11:15 (245 MB/s) - ‘tydiqa-goldp-v1.1-train.json.1’ saved [58004076/58004076]

--2021-01-25 07:11:15--  https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-dev.json
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.121.128, 74.125.126.128, 74.125.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.121.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5617409 (5.4M) [applica

#We need to preprocess the files before training

Preprocessing using fuzzy search is a very slow process. I advise that you save the preprocessed files somewhere for later use.

In [ ]:
model_name="aubmindlab/bert-base-arabertv02"

In [ ]:
!rm -rf *-pre.json
!python squad_preprocessing.py \
  --input_file "tydiqa-goldp-v1.1-train.json" \
  --output_file "tydiqa-goldp-v1.1-train-pre.json" \
  --model_name=$model_name \
  --filter_tydiqa=True

2021-01-25 07:11:26.027422: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
aubmindlab/bert-base-arabertv02
I0125 07:11:27.806138 140233621428096 preprocess.py:103] Selected Model doesn't require pre-segmentation, skipping FarasaSegmenter initialization
 49% 24648/49881 [00:05<00:07, 3411.12it/s]  WARNING:tensorflow:Could not find answer for question 'arabic-1804271180688859213-0' :
 'الإسكندر الثالث المقدوني ، المعروف بأسماء عديدة أخرى أبرزها : الإسكندر الأكبر ، و < b data - parsoid = ' { " dsr " : [1849 , 1870 , 3 , 3] } ' > الإسكندر الكبير ، و < b data - parsoid = ' { " dsr " : [1873 , 1896 , 3 , 3] } ' > الإسكندر المقدوني ، و < b data - parsoid = ' { " dsr " : [1899 , 1924 , 3 , 3] } ' > الإسكندر ذو القرنين ( باليونانية : ؛ نقحرة : ) ، هو أحد ملوك مقدونيا الإغريق ، ومن أشهر القادة العسكريين والفاتحين عبر التاريخ . ولد الإسكندر في مدينة يلا قرابة سنة 356 ق . م ، وتتلمذ على يد الفيلسوف والعالم الشهير أرسطو حتى بلغ 

In [ ]:
!python squad_preprocessing.py \
  --input_file "tydiqa-goldp-v1.1-dev.json" \
  --output_file "tydiqa-goldp-v1.1-dev-pre.json" \
  --model_name=$model_name \
  --filter_tydiqa=True

2021-01-25 07:11:53.872172: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
aubmindlab/bert-base-arabertv02
I0125 07:11:55.618276 140435043919744 preprocess.py:103] Selected Model doesn't require pre-segmentation, skipping FarasaSegmenter initialization
100% 5077/5077 [00:00<00:00, 5765.17it/s]


#Construct a custom dataset loader:


In [ ]:
#@title
%%writefile artydiqa.py
"""TODO(tydiqa): Add a description here."""

from __future__ import absolute_import, division, print_function

import json
import textwrap

import datasets


# TODO(tydiqa): BibTeX citation
_CITATION = """\
@article{tydiqa,
title   = {TyDi QA: A Benchmark for Information-Seeking Question Answering in Typologically Diverse Languages},
author  = {Jonathan H. Clark and Eunsol Choi and Michael Collins and Dan Garrette and Tom Kwiatkowski and Vitaly Nikolaev and Jennimaria Palomaki}
year    = {2020},
journal = {Transactions of the Association for Computational Linguistics}
}
"""

# TODO(tydiqa):
_DESCRIPTION = """\
TyDi QA is a question answering dataset covering 11 typologically diverse languages with 204K question-answer pairs.
The languages of TyDi QA are diverse with regard to their typology -- the set of linguistic features that each language
expresses -- such that we expect models performing well on this set to generalize across a large number of the languages
in the world. It contains language phenomena that would not be found in English-only corpora. To provide a realistic
information-seeking task and avoid priming effects, questions are written by people who want to know the answer, but
don’t know the answer yet, (unlike SQuAD and its descendents) and the data is collected directly in each language without
the use of translation (unlike MLQA and XQuAD).
"""

# _URL = "https://storage.googleapis.com/tydiqa/"

# _PRIMARY_URLS = {
#     "train": _URL + "v1.0/tydiqa-v1.0-train.jsonl.gz",
#     "dev": _URL + "v1.0/tydiqa-v1.0-dev.jsonl.gz",
# }
# _SECONDARY_URLS = {
#     "train": _URL + "v1.1/tydiqa-goldp-v1.1-train.json",
#     "dev": _URL + "v1.1/tydiqa-goldp-v1.1-dev.json",
# }

#use this for AraBERTv1 and V2
_URL = "https://storage.googleapis.com/tydiqa/"
_URL2 = "/content/"
_PRIMARY_URLS = {
    "train": _URL + "v1.0/tydiqa-v1.0-train.jsonl.gz",
    "dev": _URL + "v1.0/tydiqa-v1.0-dev.jsonl.gz",
}
_SECONDARY_URLS = {
    "train": _URL2 + "tydiqa-goldp-v1.1-train-pre.json",
    "dev": _URL2 + "tydiqa-goldp-v1.1-dev-pre.json",
}


class TydiqaConfig(datasets.BuilderConfig):

    """ BuilderConfig for Tydiqa"""

    def __init__(self, **kwargs):
        """

        Args:
            **kwargs: keyword arguments forwarded to super.
        """
        super(TydiqaConfig, self).__init__(version=datasets.Version("1.0.0", ""), **kwargs)


class Tydiqa(datasets.GeneratorBasedBuilder):
    """TODO(tydiqa): Short description of my dataset."""

    # TODO(tydiqa): Set up version.
    VERSION = datasets.Version("0.1.0")
    BUILDER_CONFIGS = [
        TydiqaConfig(
            name="primary_task",
            description=textwrap.dedent(
                """\
          Passage selection task (SelectP): Given a list of the passages in the article, return either (a) the index of
          the passage that answers the question or (b) NULL if no such passage exists.
          Minimal answer span task (MinSpan): Given the full text of an article, return one of (a) the start and end
          byte indices of the minimal span that completely answers the question; (b) YES or NO if the question requires
          a yes/no answer and we can draw a conclusion from the passage; (c) NULL if it is not possible to produce a
          minimal answer for this question."""
            ),
        ),
        TydiqaConfig(
            name="secondary_task",
            description=textwrap.dedent(
                """Gold passage task (GoldP): Given a passage that is guaranteed to contain the
          answer, predict the single contiguous span of characters that answers the question. This is more similar to
          existing reading comprehension datasets (as opposed to the information-seeking task outlined above).
          This task is constructed with two goals in mind: (1) more directly comparing with prior work and (2) providing
          a simplified way for researchers to use TyDi QA by providing compatibility with existing code for SQuAD 1.1,
          XQuAD, and MLQA. Toward these goals, the gold passage task differs from the primary task in several ways:
          only the gold answer passage is provided rather than the entire Wikipedia article;
          unanswerable questions have been discarded, similar to MLQA and XQuAD;
          we evaluate with the SQuAD 1.1 metrics like XQuAD; and
         Thai and Japanese are removed since the lack of whitespace breaks some tools.
          """
            ),
        ),
    ]

    def _info(self):
        # TODO(tydiqa): Specifies the datasets.DatasetInfo object
        if self.config.name == "primary_task":
            return datasets.DatasetInfo(
                # This is the description that will appear on the datasets page.
                description=_DESCRIPTION,
                # datasets.features.FeatureConnectors
                features=datasets.Features(
                    {
                        "passage_answer_candidates": datasets.features.Sequence(
                            {
                                "plaintext_start_byte": datasets.Value("int32"),
                                "plaintext_end_byte": datasets.Value("int32"),
                            }
                        ),
                        "question_text": datasets.Value("string"),
                        "document_title": datasets.Value("string"),
                        "language": datasets.Value("string"),
                        "annotations": datasets.features.Sequence(
                            {
                                # 'annotation_id': datasets.Value('variant'),
                                "passage_answer_candidate_index": datasets.Value("int32"),
                                "minimal_answers_start_byte": datasets.Value("int32"),
                                "minimal_answers_end_byte": datasets.Value("int32"),
                                "yes_no_answer": datasets.Value("string"),
                            }
                        ),
                        "document_plaintext": datasets.Value("string"),
                        # 'example_id': datasets.Value('variant'),
                        "document_url": datasets.Value("string")
                        # These are the features of your dataset like images, labels ...
                    }
                ),
                # If there's a common (input, target) tuple from the features,
                # specify them here. They'll be used if as_supervised=True in
                # builder.as_dataset.
                supervised_keys=None,
                # Homepage of the dataset for documentation
                homepage="https://github.com/google-research-datasets/tydiqa",
                citation=_CITATION,
            )
        elif self.config.name == "secondary_task":
            return datasets.DatasetInfo(
                description=_DESCRIPTION,
                features=datasets.Features(
                    {
                        "id": datasets.Value("string"),
                        "title": datasets.Value("string"),
                        "context": datasets.Value("string"),
                        "question": datasets.Value("string"),
                        "answers": datasets.features.Sequence(
                            {
                                "text": datasets.Value("string"),
                                "answer_start": datasets.Value("int32"),
                            }
                        ),
                    }
                ),
                # No default supervised_keys (as we have to pass both question
                # and context as input).
                supervised_keys=None,
                homepage="https://github.com/google-research-datasets/tydiqa",
                citation=_CITATION,
            )

    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        # TODO(tydiqa): Downloads the data and defines the splits
        # dl_manager is a datasets.download.DownloadManager that can be used to
        # download and extract URLs
        primary_downloaded = dl_manager.download_and_extract(_PRIMARY_URLS)
        secondary_downloaded = dl_manager.download_and_extract(_SECONDARY_URLS)
        if self.config.name == "primary_task":
            return [
                datasets.SplitGenerator(
                    name=datasets.Split.TRAIN,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": primary_downloaded["train"]},
                ),
                datasets.SplitGenerator(
                    name=datasets.Split.VALIDATION,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": primary_downloaded["dev"]},
                ),
            ]
        elif self.config.name == "secondary_task":
            return [
                datasets.SplitGenerator(
                    name=datasets.Split.TRAIN,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": secondary_downloaded["train"]},
                ),
                datasets.SplitGenerator(
                    name=datasets.Split.VALIDATION,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": secondary_downloaded["dev"]},
                ),
            ]

    def _generate_examples(self, filepath):
        """Yields examples."""
        # TODO(tydiqa): Yields (key, example) tuples from the dataset
        if self.config.name == "primary_task":
            with open(filepath, encoding="utf-8") as f:
                for id_, row in enumerate(f):
                    data = json.loads(row)
                    passages = data["passage_answer_candidates"]
                    end_byte = [passage["plaintext_end_byte"] for passage in passages]
                    start_byte = [passage["plaintext_start_byte"] for passage in passages]
                    title = data["document_title"]
                    lang = data["language"]
                    question = data["question_text"]
                    annotations = data["annotations"]
                    # annot_ids = [annotation["annotation_id"] for annotation in annotations]
                    yes_no_answers = [annotation["yes_no_answer"] for annotation in annotations]
                    min_answers_end_byte = [
                        annotation["minimal_answer"]["plaintext_end_byte"] for annotation in annotations
                    ]
                    min_answers_start_byte = [
                        annotation["minimal_answer"]["plaintext_start_byte"] for annotation in annotations
                    ]
                    passage_cand_answers = [
                        annotation["passage_answer"]["candidate_index"] for annotation in annotations
                    ]
                    doc = data["document_plaintext"]
                    # example_id = data["example_id"]
                    url = data["document_url"]
                    yield id_, {
                        "passage_answer_candidates": {
                            "plaintext_start_byte": start_byte,
                            "plaintext_end_byte": end_byte,
                        },
                        "question_text": question,
                        "document_title": title,
                        "language": lang,
                        "annotations": {
                            # 'annotation_id': annot_ids,
                            "passage_answer_candidate_index": passage_cand_answers,
                            "minimal_answers_start_byte": min_answers_start_byte,
                            "minimal_answers_end_byte": min_answers_end_byte,
                            "yes_no_answer": yes_no_answers,
                        },
                        "document_plaintext": doc,
                        # 'example_id': example_id,
                        "document_url": url,
                    }
        elif self.config.name == "secondary_task":
            with open(filepath, encoding="utf-8") as f:
                data = json.load(f)
                for article in data["data"]:
                    title = article.get("title", "").strip()
                    for paragraph in article["paragraphs"]:
                        context = paragraph["context"].strip()
                        for qa in paragraph["qas"]:
                            question = qa["question"].strip()
                            id_ = qa["id"]
                            if "arabic" not in id_:
                              continue
                            answer_starts = [answer["answer_start"] for answer in qa["answers"] if answer["answer_start"] != -1]
                            answers = [answer["text"].strip() for answer in qa["answers"] if answer["answer_start"] != -1]
                            if len(answers) == 0 or len(answer_starts)==0:
                              print("question skipped")
                              continue
                            assert len(answer_starts)==len(answers)
                            # Features currently used are "context", "question", and "answers".
                            # Others are extracted here for the ease of future expansions.
                            yield id_, {
                                "title": title,
                                "context": context,
                                "question": question,
                                "id": id_,
                                "answers": {
                                    "answer_start": answer_starts,
                                    "text": answers,
                                },
                            }

Overwriting artydiqa.py


#Run Pytorch Training

In [ ]:
#!python run_qa.py --help

In [ ]:
model_name="aubmindlab/araelectra-base-discriminator"

In [ ]:
!python run_qa.py \
  --model_name_or_path $model_name \
  --dataset_name artydiqa.py \
  --dataset_config_name "secondary_task" \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 8 \
  --gradient_accumulation_steps 1 \
  --learning_rate 3e-5 \
  --num_train_epochs 4 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ./run \
  --n_best_size 20 \
  --evaluation_strategy epoch \
  --save_steps 20160 \
  --overwrite_output_dir \
  --seed 42 \
  --warmup_steps 500 \
  # --load_best_model_at_end \
  # --metric_for_best_model exact_match \
  # --greater_is_better True

2021-01-25 13:36:35.420432: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
01/25/2021 13:36:36 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
01/25/2021 13:36:36 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=./run, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=4.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=500, logging_dir=runs/Jan25_13-36-36_a327e920aa13, logging_first_step=False, logging_steps=500, save_steps=20160, save_total_limit=Non